# Create wflow evaluation period config file using calibrated parameters
## Based on best performing ksathorfrac values

In [ ]:
from glob import glob
from pathlib import Path

import toml
import pandas as pd

# Set Paths

In [ ]:
# Set Paths
ROOT = Path("/gpfs/work1/0/wtrcycle/users/jaerts/camels_uk/")
MODELS = Path(f'{ROOT}/wflow/data/')
RESULTS = Path(f"{ROOT}/results/")

template = f"{ROOT}/aux_data/Wflow_SBM/template_evaluation.toml"

# Load calibration results

In [ ]:
# Load calibration file
calibration_file = f"{RESULTS}/wflow_sbm/wflow_calibration_objective_function_overview.csv"
df = pd.read_csv(calibration_file, index_col='basin_id')

basin_ids = df.index.to_list()

In [ ]:
len(basin_ids)

# Create config files based on calibrated parameters

In [ ]:
# Evaluation Period
for basin_id in basin_ids:
    calibration_values = int(df.loc[int(basin_id)].ksathorfrac)  
    
    config_file = toml.load(template)
    config_file['loglevel'] = "debug"      
    config_file['path_log'] = f"log_evaluation.txt"    
    
    # Add timeperiod
    config_file['starttime'] = '2008-01-01T00:00:00'
    config_file['endtime'] = '2015-12-31T00:00:00'
    
    # Add file location
    config_file['input']['path_forcing'] = f'ceh-gear_chess_camels-gb_{basin_id}_2000_2017.nc'
    config_file['input']['path_static'] = f'staticmaps_ksathorfrac_{calibration_values}.nc'

    # Add output
    config_file['state']['path_output'] = f'evaluation/outstate/outstates.nc'
    config_file['output']['path'] = f'evaluation/output.nc'
    config_file['csv']['path'] = f'evaluation/output.csv'
   
    with open(f'{MODELS}/{basin_id}/wflow_sbm_evaluation.toml', 'w') as f:
        toml.dump(config_file, f)